# Premier exemple de régression: prédire l'âge

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Charger les données
data_census_individuals = pd.read_parquet("data/data_census_individuals.parquet")

In [ ]:
# Échantillonner les données (1/200)
data_sample = data_census_individuals.sample(frac=1/200, random_state=123)

In [ ]:
# Supprimer les variables liées à l'âge et celles avec trop de modalités
columns_to_remove = ['AGER20', 'AGEREV', 'AGEREVQ', 'ANAI', 'TRIRIS', 'IRIS', 
                     'DNAI', 'DEPT', 'ARM', 'CANTVILLE', 'NUMMI', 'IPONDI', 'MODV']
data_clean = data_sample.drop(columns=columns_to_remove)

In [ ]:
# Séparer les features (X) et la variable cible (y)
X = data_clean.drop(columns=['AGED'])
y = data_clean['AGED']

In [ ]:
# Encodage "one-hot" des colonnes catégoriques
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Identifier les colonnes catégoriques
categorical_cols = X.select_dtypes(include=['category']).columns

# Appliquer le One-Hot Encoding sur les colonnes catégoriques
X_encoded = encoder.fit_transform(X[categorical_cols])

# Convertir les colonnes encodées en DataFrame avec les noms des catégories
X_encoded = pd.DataFrame(
    X_encoded, 
    index=X.index, 
    columns=encoder.get_feature_names_out(categorical_cols)
)

# Sélectionner les colonnes numériques
numeric_cols = X.select_dtypes(exclude=['category'])

# Concaténer les colonnes numériques et encodées uniquement si les colonnes numériques ne sont pas vides
if not numeric_cols.empty:
    X = pd.concat([numeric_cols, X_encoded], axis=1)
else:
    X = X_encoded

# Vérification finale
print(f"Dimensions finales après encodage : X={X.shape}")
print("Aperçu de X :")
print(X.head())

# Vérifier les dimensions après division
print(f"Dimensions après division : X={X.shape}")

In [ ]:
# Dimensions avant et après l'encodage
print(f"Dimensions avant encodage : {X.shape[0]} lignes, {categorical_cols.shape[0]} colonnes catégoriques")


In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Modèle Random Forest
rf_model = RandomForestRegressor(
    n_estimators=500,
    max_features='sqrt',
    min_samples_leaf=5,
    random_state=123,
    n_jobs=-1
)

In [ ]:
# Entraînement du modèle
print("⏳ Entraînement du modèle Random Forest...")
rf_model.fit(X_train, y_train)

In [ ]:
# Prédictions
y_pred = rf_model.predict(X_test)

In [ ]:
# Évaluation : RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

In [ ]:
# Visualisation des résultats : Âge réel vs Âge prédit
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred, alpha=0.5, color="blue")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Âge réel")
plt.ylabel("Âge prédit")
plt.title("Âge réel vs Âge prédit")
plt.show()

In [ ]:
# Importance des variables (Mean Decrease in Impurity)
importances = rf_model.feature_importances_
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
# Visualisation des variables les plus importantes
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Variable'], importance_df['Importance'], color='steelblue')
plt.xlabel("Importance")
plt.ylabel("Variable")
plt.title("Importance des variables (Mean Decrease in Impurity)")
plt.gca().invert_yaxis()  # Inverser l'ordre des variables
plt.show()

In [ ]:
# Visualisation des n variables les plus importantes

def plot_top_n_variables(importance_df, top_n=10):
    """
    Visualise les n variables les plus importantes à partir d'un DataFrame d'importance.

    Parameters:
    - importance_df (pd.DataFrame): Un DataFrame contenant les colonnes 'Variable' et 'Importance'.
    - top_n (int): Le nombre de variables les plus importantes à afficher.
    """
    # Trier les variables par importance décroissante
    top_variables = importance_df.sort_values(by='Importance', ascending=False).head(top_n)
    
    # Création du graphique
    plt.figure(figsize=(10, 8))
    plt.barh(top_variables['Variable'], top_variables['Importance'], color='steelblue')
    plt.xlabel("Importance")
    plt.ylabel("Variable")
    plt.title(f"Top {top_n} variables les plus importantes (Mean Decrease in Impurity)")
    plt.gca().invert_yaxis()  # Inverser l'ordre des variables
    plt.show()

# Exemple d'utilisation
plot_top_n_variables(importance_df, top_n=15)

In [ ]:
# Validation croisée (10-fold)
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores).mean()
print(f"RMSE moyenne (validation croisée) : {cv_rmse:.2f}")

In [ ]:
# Importance des variables basée sur la permutation
perm_importance = permutation_importance(rf_model, X_test, y_test, n_repeats=10, random_state=123, n_jobs=-1)
perm_importance_df = pd.DataFrame({
    'Variable': X.columns,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

In [ ]:
# Visualisation des variables les plus importantes (permutation)
plt.figure(figsize=(10, 8))
plt.barh(perm_importance_df['Variable'], perm_importance_df['Importance'], color='steelblue')
plt.xlabel("Importance (Permutation)")
plt.ylabel("Variable")
plt.title("Importance des variables (Permutation)")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Afficher les variables les plus importantes
print(perm_importance_df.head(10))